In [1]:
import pandas as pd
df = pd.read_csv("listing.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5229 entries, 0 to 5228
Data columns (total 7 columns):
Unnamed: 0     5229 non-null int64
ticker         5229 non-null object
name           5229 non-null object
website        5228 non-null object
sector         5221 non-null object
industry       5221 non-null object
description    5229 non-null object
dtypes: int64(1), object(6)
memory usage: 286.0+ KB


In [2]:
#only grab noun as key words
from nltk.corpus import wordnet as wn
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}

def get_keywords(text,company):
    from gensim import corpora
    from gensim.models.ldamodel import LdaModel
    from gensim.parsing.preprocessing import STOPWORDS
    ignore = ['company','segment','markets','market','department','american','customers','provides','offers','serves','sells','products','services','united','states','january','february',
             'march','april','may','june','july','august','september','october','november','december','known','new']
    company_list = company.lower().split(' ')
    texts = [[word for word in text.lower().split()
            if word not in STOPWORDS and word.isalnum() and not word in company_list and not word.lower() in ignore
             and not word.isdigit() and word in nouns]  
            ]
    
    if [] not in texts:
        dictionary = corpora.Dictionary(texts) 
        corpus = [dictionary.doc2bow(text) for text in texts]
        lda = LdaModel(corpus, id2word=dictionary, num_topics=5, passes=10)
        from operator import itemgetter
        i = sorted(lda.get_document_topics(corpus[0],minimum_probability=0,per_word_topics=False),key=itemgetter(1),reverse=True)[0][0]
        # lda.print_topic(topicno=i) 
        topic_list = lda.show_topic(i)[:5]
        key_list = list()
        for tup in topic_list:
            key_list.append(tup[0])
        key_words = ', '.join(str(i) for i in key_list)
    else:
        key_words = None
    return key_words
 
    

In [3]:
df['key_words'] = df.apply(lambda row: get_keywords(row['description'],row['name']),axis = 1)

In [4]:
df = df[df['key_words'].notnull()]

In [5]:
# there're only 2 missing value
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5227 entries, 0 to 5228
Data columns (total 8 columns):
Unnamed: 0     5227 non-null int64
ticker         5227 non-null object
name           5227 non-null object
website        5226 non-null object
sector         5219 non-null object
industry       5219 non-null object
description    5227 non-null object
key_words      5227 non-null object
dtypes: int64(1), object(7)
memory usage: 367.5+ KB


In [6]:
df.to_csv('list_with_keywords.csv')

In [2]:
#summarization
import pandas as pd
df = pd.read_csv('update3.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5224 entries, 0 to 5223
Data columns (total 12 columns):
ticker         5224 non-null object
name           5224 non-null object
website        5223 non-null object
sector         5216 non-null object
industry       5216 non-null object
description    5224 non-null object
key_words      5224 non-null object
facebook       2451 non-null object
twitter        2766 non-null object
linkedin       2775 non-null object
youtube        1756 non-null object
instagram      1137 non-null object
dtypes: object(12)
memory usage: 489.8+ KB


In [3]:
group_by_sector = df.groupby(['sector']).size().to_frame()
# group_by_sector.plot(kind = 'bar',legend=False)

In [4]:
group_by_sector

,0
sector,
Basic Materials,237
Communication Services,240
Consumer Cyclical,478
Consumer Defensive,197
Energy,318
Financial,2
Financial Services,1235
Healthcare,963
Industrial Goods,1
